In [ ]:
# parameters

In [ ]:
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise
import Downloads

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "StatsBase",
    "ProgressMeter",
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
genbank_metadata = Mycelia.load_genbank_metadata();

In [ ]:
refseq_metadata = Mycelia.load_refseq_metadata();

In [ ]:
# seems plenty large enough?
# nt_viruses - 60Gb
# # IMG/VR - 25Gb
# prokaryotes (bacteria + archaea) nt_prok.fna.gz - 69G
# nt_other - 1.6Gb

In [ ]:
genbank_representative_non_excluded = 
genbank_metadata[
    map(x -> x in ["reference genome", "representative genome"], genbank_metadata[!, "refseq_category"]) .&
    (genbank_metadata[!, "excluded_from_refseq"] .== "na"), :
];

In [ ]:
# protozoa - genbank representative non-excluded
# 20G     protozoa/
group = "protozoa"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = genbank_representative_non_excluded[genbank_representative_non_excluded[!, "group"] .== group, "ftp_path"]
# 12 minutes
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_files(files=fasta_list, file=outfasta)
end

In [ ]:
# fungi - genbank representative non-excluded
# 57G     fungi/
group = "fungi"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = genbank_representative_non_excluded[genbank_representative_non_excluded[!, "group"] .== group, "ftp_path"]
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_fasta_files(fasta_files=fasta_list, fasta_file=outfasta)
end

In [ ]:
refseq_representative = refseq_metadata[map(x -> x in ["reference genome", "representative genome"], refseq_metadata[!, "refseq_category"]), :];

In [ ]:
# plant - refseq representative
# 52G     plant/
group = "plant"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = refseq_representative[refseq_representative[!, "group"] .== group, "ftp_path"]
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_fasta_files(fasta_files=fasta_list, fasta_file=outfasta)
end

In [ ]:
# invertebrate - refseq representative
# 75G     invertebrate/
group = "invertebrate"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = refseq_representative[refseq_representative[!, "group"] .== group, "ftp_path"]
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_fasta_files(fasta_files=fasta_list, fasta_file=outfasta)
    # special case just for bad invertebrate seqs with duplicate identifiers
    Mycelia.add_bioconda_env("seqkit")
    deduped_fasta = replace(outfasta, Mycelia.FASTA_REGEX => ".deduped.fna.gz")
    run(`$(Mycelia.CONDA_RUNNER) run --live-stream -n seqkit seqkit rmdup -i -o $(deduped_fasta) $(outfasta)`)
    mv(deduped_fasta, outfasta)
end

In [ ]:
# 164Gb
# vertebrate - other - refseq representative
group = "vertebrate_other"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = refseq_representative[refseq_representative[!, "group"] .== group, "ftp_path"]
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_fasta_files(fasta_files=fasta_list, fasta_file=outfasta)
end

In [ ]:
# vertebrate - mammal - refseq representative
# 168Gb
group = "vertebrate_mammalian"
outdir = mkpath(joinpath("$(homedir())/workspace/blastdb/$(group)"))
ftps = refseq_representative[refseq_representative[!, "group"] .== group, "ftp_path"]
ProgressMeter.@showprogress for ftp in ftps
    try
        Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
    catch e
        display(e)
    end
end
fasta_list = filter(x -> occursin(Mycelia.FASTA_REGEX, x), readdir(outdir, join=true))
outfasta = outdir * ".fna.gz"
if !isfile(outfasta)
    Mycelia.concatenate_fasta_files(fasta_files=fasta_list, fasta_file=outfasta)
end

In [ ]:
# # 3 days to download like this!
# ProgressMeter.@showprogress for row in DataFrames.eachrow(subset)
#     Mycelia.ncbi_genome_download_accession(
#         accession=row["#assembly_accession"],
#         outdir=outdir
#     )
# end